In [1]:
%matplotlib inline
import numpy as np
import numpy.ma as ma
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns
import timeit
import line_profiler

import datetime

from io import StringIO
from pandas.api.types import CategoricalDtype

import pandas._testing as tm

## returns df from string data
def g(data):
    return pd.read_csv(StringIO(data))

In [23]:
d="""date,data
2019-01-03,10
2019-01-04,20
2019-05-21,30
2019-05-22,40
2020-10-15,50
2020-10-16,60
2021-04-04,70
2021-04-05,80"""
df=g(d)
df
df['date'] = pd.to_datetime(df.date)
df

,date,data
0,2019-01-03,10
1,2019-01-04,20
2,2019-05-21,30
3,2019-05-22,40
4,2020-10-15,50
5,2020-10-16,60
6,2021-04-04,70
7,2021-04-05,80


In [31]:
#df['year'] = pd.to_datetime(df.date).dt.isocalendar()
df[['year','week']] = pd.to_datetime(df.date).dt.isocalendar()[['year', 'week']]
pd.pivot_table(data = df,index= 'year', columns = 'week',values = 'data', aggfunc = np.mean )

week,1,13,14,21,42
year,,,,,
2019,15.0,NaN,NaN,35.0,NaN
2020,NaN,NaN,NaN,NaN,55.0
2021,NaN,70.0,80.0,NaN,NaN


In [7]:
df=pd.DataFrame({
    'mydate1':['17JUL2019:00:00:00','18MAR2019:00:00:00',
               '27MAR2019:00:00:00','04SEP2018:00:00:00',
              '07NOV2017:00:00:00','23FEB2018:00:00:00'],
    'mydate2':'31Dec2019',
    'curmth':[0,0,1,4,11,14]
})
df.mydate1 = pd.to_datetime(df.mydate1, format='%d%b%Y:%H:%M:%S')
df.mydate2 = pd.to_datetime(df.mydate2, format='%d%b%Y', )
df

,mydate1,mydate2,curmth
0,2019-07-17,2019-12-31,0
1,2019-03-18,2019-12-31,0
2,2019-03-27,2019-12-31,1
3,2018-09-04,2019-12-31,4
4,2017-11-07,2019-12-31,11
5,2018-02-23,2019-12-31,14


In [8]:
mnth_diff = (df.mydate2 - df.mydate1)//np.timedelta64(1,'M')
df['num_mths']=mnth_diff-df.curmth-1
df

,mydate1,mydate2,curmth,num_mths
0,2019-07-17,2019-12-31,0,4
1,2019-03-18,2019-12-31,0,8
2,2019-03-27,2019-12-31,1,7
3,2018-09-04,2019-12-31,4,10
4,2017-11-07,2019-12-31,11,13
5,2018-02-23,2019-12-31,14,7


In [41]:
d="""Disease,HeartRate,State,MonthStart,MonthEnd
Covid,89,Texas,2020-02-28,2020-03-31
Covid,91,Texas,2020-03-31,2020-04-30
Covid,87,Texas,2020-07-31,2020-08-30
Cancer,90,Texas,2020-02-28,2020-03-31 
Cancer,88,Florida,2020-03-31,2020-04-30
Covid,89,Florida,2020-02-28,2020-03-31
Covid,87,Florida,2020-03-31,2020-04-30
Flu,90,Florida,2020-02-28,2020-03-31"""
df=g(d)
df.MonthEnd = pd.to_datetime(df.MonthEnd)
df

,Disease,HeartRate,State,MonthStart,MonthEnd
0,Covid,89,Texas,2020-02-28,2020-03-31
1,Covid,91,Texas,2020-03-31,2020-04-30
2,Covid,87,Texas,2020-07-31,2020-08-30
3,Cancer,90,Texas,2020-02-28,2020-03-31
4,Cancer,88,Florida,2020-03-31,2020-04-30
5,Covid,89,Florida,2020-02-28,2020-03-31
6,Covid,87,Florida,2020-03-31,2020-04-30
7,Flu,90,Florida,2020-02-28,2020-03-31


In [113]:
df=pd.DataFrame({
    'timestamp':pd.date_range('2020.01.01', periods=6, freq='30S'),
    'val':[1,2,3,0,0,4]
})
df=df.set_index('timestamp')
df.val = df.val.replace(0, np.nan)
df = df.resample('60s').agg(['mean','last'])
df

val     
                    mean last
timestamp                    
2020-01-01 00:00:00  1.5  2.0
2020-01-01 00:01:00  3.0  3.0
2020-01-01 00:02:00  4.0  4.0

In [186]:
y_pred = np.array([1])
pd.Series(y_pred).to_frame()



0    1
dtype: int64

In [187]:
idx=pd.date_range('2013.01.07', periods=2)
ts = pd.Series(['10,559.01', '10,508.06'],
         index=idx)
ts

2013-01-07    10,559.01
2013-01-08    10,508.06
Freq: D, dtype: object

In [188]:
d="""company,product,security_level,rating,rating_level,first_release,last_release,score
google,mobile,minor,TRUE,critical,04/11/2020,now,0.5
google,os,moderate,FALSE,medium,09/05/2019,03/20/2021,0.009
google,os,minor,FALSE,low,09/04/2019,05/11/2020,19
google,tv,severe,TRUE,high,08/11/2020,03/04/2021,
google,mobile,supersevere,FALSE,medium,04/06/2015,08/19/2020,2.4
google,mobile,minor,FALSE,high,08/08/2019,08/19/2020,1.3
apple,iphone,minor,TRUE,low,02/03/2020,now,3
apple,iphone,material,TRUE,medium,01/21/2018,03/04/2021,6
apple,iwatch,material,FALSE,low,04/11/2015,08/13/2020,8
apple,iphone,material,TRUE,medium,10/20/2020,03/19/2021,5
dell,laptop,minor,FALSE,low,01/05/2021,03/20/2021,1"""
df=g(d)
df

,company,product,security_level,rating,rating_level,first_release,last_release,score
0,google,mobile,minor,True,critical,04/11/2020,now,0.500
1,google,os,moderate,False,medium,09/05/2019,03/20/2021,0.009
2,google,os,minor,False,low,09/04/2019,05/11/2020,19.000
3,google,tv,severe,True,high,08/11/2020,03/04/2021,NaN
4,google,mobile,supersevere,False,medium,04/06/2015,08/19/2020,2.400
5,google,mobile,minor,False,high,08/08/2019,08/19/2020,1.300
6,apple,iphone,minor,True,low,02/03/2020,now,3.000
7,apple,iphone,material,True,medium,01/21/2018,03/04/2021,6.000
8,apple,iwatch,material,False,low,04/11/2015,08/13/2020,8.000
9,apple,iphone,material,True,medium,10/20/2020,03/19/2021,5.000


In [189]:
df2 = pd.DataFrame([{"a":1.0,"b":2,"c":3,"d":4}]).set_index(["c","d"])
df2

,,a,b
c,d,,
3,4,1.0,2


In [249]:
data = StringIO('''\
Values
1992-08-27 07:00:00,    8.0,    7.0,    A
1992-08-27 08:00:00,    8.2,    7.0,    A
1992-08-27 09:30:00,    8.5,    7.5,    A
1992-08-27 11:00:00,    8.8,    7.5,    A
1992-08-27 14:30:00,    9.0,    7.0,    A
1992-08-27 16:00:00,    9.6,    6.5,    A
1992-08-27 18:30:00,    9.6,    7.0,    A
1992-08-27 21:00:00,    9.8,    7.1,    A
1992-08-27 23:30:00,    0.0,    7.2,    A
1992-08-28 01:00:00,    8.0,    7.3,    A
1992-08-28 02:00:00,    8.2,    7.4,    A
1992-08-28 03:30:00,    8.5,    7.6,    A
1992-08-28 11:00:00,    8.8,    7.5,    A
1992-08-28 14:30:00,    9.0,    7.1,    A
1992-08-28 16:00:00,    9.6,    6.2,    A
1992-08-28 18:30:00,    9.6,    7.7,    A
1992-08-28 21:00:00,    9.8,    4.0,    A
1992-08-28 23:30:00,    0.0,    2.0,    A
1992-08-27 01:30:00,    29.2,    29.0,    B
1992-08-27 08:00:00,    29.2,    29.0,    B
1992-08-27 08:30:00,    29.2,    29.0,    B
1992-08-27 09:00:00,    29.2,    29.0,    B
1992-08-27 09:30:00,    30.0,    37.0,    B
1992-08-27 10:30:00,    24.6,    37.0,    B
1992-08-27 11:00:00,    24.8,    37.0,    B
1992-08-27 11:30:00,    24.8,    37.0,    B
''')
s = pd.read_csv(data, squeeze=True)
s = pd.DataFrame(s)
s = s.reset_index()
s.columns = ['datetime', 'value1', 'value2', 'name']
s.head()


,datetime,value1,value2,name
0,1992-08-27 07:00:00,8.0,7.0,A
1,1992-08-27 08:00:00,8.2,7.0,A
2,1992-08-27 09:30:00,8.5,7.5,A
3,1992-08-27 11:00:00,8.8,7.5,A
4,1992-08-27 14:30:00,9.0,7.0,A


In [9]:
df = pd.DataFrame({'col1': [1, 1, 1, 2, 2, 2, 3, 3, 3], 'col2': ['a', 'b', 'c', 'A', 'C', 'B', 'red', 'blue', 'greed']})
df

,col1,col2
0,1,a
1,1,b
2,1,c
3,2,A
4,2,C
5,2,B
6,3,red
7,3,blue
8,3,greed


In [13]:
df['col2'] = df.groupby('col1', as_index=False).col2.transform(np.random.permutation)
df

,col1,col2
0,1,c
1,1,b
2,1,a
3,2,B
4,2,A
5,2,C
6,3,red
7,3,greed
8,3,blue


In [15]:
d="""BKG_PRODTN_DT|POL_EXP_INFO|BKD_MAJ_LINE_CD|POL_SYM_CD|AMT|bkd_amt
20210319,20210319|100,100|A001,A001|NVA,NVA|14.26,14.26|10,13.97
20210319,20210319|101,101|A111,A112|NVA,NVA|63,64.17|62.89,62.89"""
df=pd.read_csv(StringIO(d), sep='|')
df

,BKG_PRODTN_DT,POL_EXP_INFO,BKD_MAJ_LINE_CD,POL_SYM_CD,AMT,bkd_amt
0,"20210319,20210319","100,100","A001,A001","NVA,NVA","14.26,14.26","10,13.97"
1,"20210319,20210319","101,101","A111,A112","NVA,NVA","63,64.17","62.89,62.89"


In [36]:
df.replace(r"^(.*),\1$", r"\1", regex=True)

,BKG_PRODTN_DT,POL_EXP_INFO,BKD_MAJ_LINE_CD,POL_SYM_CD,AMT,bkd_amt
0,20210319,100,A001,NVA,14.26,"10,13.97"
1,20210319,101,"A111,A112",NVA,"63,64.17",62.89


In [40]:
assets = ['AAPL','AAPL','AAPL','AAPL','AAPL','MSFT','MSFT','MSFT','MSFT','MSFT','AMZN','AMZN','AMZN','AMZN','AMZN',]
dates  = ['05/01/2021','05/02/2021','05/03/2021','05/04/2021','05/05/2021','05/01/2021','05/02/2021','05/03/2021','05/04/2021','05/05/2021','05/01/2021','05/02/2021','05/03/2021','05/04/2021','05/05/2021']
prices = range(1, 16)

test_data = pd.DataFrame(index=dates)
test_data['Asset'] = assets
test_data['Price'] = prices

test_data

,Asset,Price
05/01/2021,AAPL,1
05/02/2021,AAPL,2
05/03/2021,AAPL,3
05/04/2021,AAPL,4
05/05/2021,AAPL,5
05/01/2021,MSFT,6
05/02/2021,MSFT,7
05/03/2021,MSFT,8
05/04/2021,MSFT,9
05/05/2021,MSFT,10


In [49]:
df = test_data.pivot(columns='Asset')
df

Price          
Asset       AAPL AMZN MSFT
05/01/2021     1   11    6
05/02/2021     2   12    7
05/03/2021     3   13    8
05/04/2021     4   14    9
05/05/2021     5   15   10

In [67]:
df = test_data.pivot(columns='Asset').droplevel(0,1).rename_axis(None, axis='columns')
df

,AAPL,AMZN,MSFT
05/01/2021,1,11,6
05/02/2021,2,12,7
05/03/2021,3,13,8
05/04/2021,4,14,9
05/05/2021,5,15,10


In [68]:
value_list = ["DDD","MMM","AAPL","MSFT","AMZN","TSLA"]
test_data2 = test_data[test_data['Asset'].isin(value_list)]
def generate_stock_price_dataframe():
    price_dataframe = pd.DataFrame()
    for stock in test_data2['Asset'].unique():
        data = pd.DataFrame(index = test_data2.index.unique())
        data[stock] = pd.DataFrame(test_data2.query("Asset==@stock")['Price'])
        price_dataframe = pd.concat([price_dataframe,data],axis=1)

    stock_price_data  = price_dataframe
    return stock_price_data
df = generate_stock_price_dataframe()
df

,AAPL,MSFT,AMZN
05/01/2021,1,6,11
05/02/2021,2,7,12
05/03/2021,3,8,13
05/04/2021,4,9,14
05/05/2021,5,10,15


In [247]:
d="""date,time,open,high,low,close,volume
2021-05-06,04:08:00,9150090.0,9150090.0,9125001.0,9130000.0,9.015642
2021-05-06,04:09:00,9140000.0,9145000.0,9125012.0,9134068.0,3.121043
2021-05-06,04:10:00,9133882.0,9133882.0,9125002.0,9132999.0,5.536345
2021-05-06,04:11:00,9132999.0,9135013.0,9131000.0,9132999.0,5.880620"""
d1="""2021-05-06,04:08:00,9150090.0,9150090.0,9125001.0,9130000.0,9.015642
2021-05-06,04:09:00,9140000.0,9145000.0,9125012.0,9134068.0,3.121043
2021-05-06,04:10:00,9133882.0,9133882.0,9125002.0,9132999.0,5.536345
2021-05-06,04:11:00,9132999.0,9135013.0,9131000.0,9132999.0,5.880620"""
df1=pd.read_csv(StringIO(d))
df1.to_csv('te.csv')
df2=pd.read_csv('te.csv')
df_stream = df1.append(df2).reset_index(drop=True)
df_stream

,date,time,open,high,low,close,volume,Unnamed: 0
0,2021-05-06,04:08:00,9150090.0,9150090.0,9125001.0,9130000.0,9.015642,NaN
1,2021-05-06,04:09:00,9140000.0,9145000.0,9125012.0,9134068.0,3.121043,NaN
2,2021-05-06,04:10:00,9133882.0,9133882.0,9125002.0,9132999.0,5.536345,NaN
3,2021-05-06,04:11:00,9132999.0,9135013.0,9131000.0,9132999.0,5.880620,NaN
4,2021-05-06,04:08:00,9150090.0,9150090.0,9125001.0,9130000.0,9.015642,0.0
5,2021-05-06,04:09:00,9140000.0,9145000.0,9125012.0,9134068.0,3.121043,1.0
6,2021-05-06,04:10:00,9133882.0,9133882.0,9125002.0,9132999.0,5.536345,2.0
7,2021-05-06,04:11:00,9132999.0,9135013.0,9131000.0,9132999.0,5.880620,3.0


In [248]:
df = df.drop_duplicates(subset=['date'])
df

,date,time,open,high,low,close,volume
0,2021-05-06,04:08:00,9150090.0,9150090.0,9125001.0,9130000.0,9.015642


In [74]:
d="""dates,A,B,C
2005-01-01,1.0,2.0,1.0
2005-01-02,2.0,1.0,1.0
2005-01-04,3.0,0.0,1.0"""
df=g(d)
df.dates = pd.to_datetime(df.dates)
df = df.set_index('dates')
df

,A,B,C
dates,,,
2005-01-01,1.0,2.0,1.0
2005-01-02,2.0,1.0,1.0
2005-01-04,3.0,0.0,1.0


In [86]:
import datetime as dt
from scipy import stats
df['date_ordinal'] = pd.to_datetime(df.index).map(dt.datetime.toordinal)
slope, intercept, r_value, p_value, std_err = stats.linregress(df['date_ordinal'], df.A)
slope

0.6428571428571428

In [103]:
d="""date,deaths
2021-05-04,134
2021-05-03,120
2021-05-02,120
2021-04-30,119
2021-04-29,114"""
df=g(d)
df

,date,deaths
0,2021-05-04,134
1,2021-05-03,120
2,2021-05-02,120
3,2021-04-30,119
4,2021-04-29,114


In [104]:
df.set_index('date', inplace=True)
df = df.sort_index(ascending=False)
#df['death_diff'] = df.deaths.diff().abs()
df_plot_2 = df['deaths'].shift() - df['deaths']
df['deaths_by_day'] = df_plot_2
df

,deaths,deaths_by_day
date,,
2021-05-04,134,NaN
2021-05-03,120,14.0
2021-05-02,120,0.0
2021-04-30,119,1.0
2021-04-29,114,5.0


In [179]:
d="""month,day,time,time-zone
May,18,3:45,PDT
May,22,6:23,EDT
May,30,22:30,CDT
June,1,18:00,PDT
June,10,1:32,MDT
June,27,12:00,EDT"""
df=g(d)
df
df['month'] = df.month.str[:3]
df.insert(0,'year','2021')
tz_map={'PDT':'US/Pacific', 'EDT':'US/Eastern','CDT':'US/Central', 'MDT':'US/Mountain'}
df['date'] = df[df.columns[:4]].apply(
    lambda x: ':'.join(x.dropna().astype(str)),
    axis=1
)
df['tz'] = df['time-zone'].map(tz_map)
df

,year,month,day,time,time-zone,date,tz
0,2021,May,18,3:45,PDT,2021:May:18:3:45,US/Pacific
1,2021,May,22,6:23,EDT,2021:May:22:6:23,US/Eastern
2,2021,May,30,22:30,CDT,2021:May:30:22:30,US/Central
3,2021,Jun,1,18:00,PDT,2021:Jun:1:18:00,US/Pacific
4,2021,Jun,10,1:32,MDT,2021:Jun:10:1:32,US/Mountain
5,2021,Jun,27,12:00,EDT,2021:Jun:27:12:00,US/Eastern


In [216]:
df1 = pd.DataFrame({
    'name':['ab', 'cd','ef'],
    'email':['abc@g', 'cde@x','efg@y'],
    'prof':['doc','engg','dan']
})
df2 = pd.DataFrame({
    'name':['ab','cd','ef'],
    'email':['abc@g','cd@x','efg@y'],
    'prof':['pro','doc','dan']
})
diff =df1.merge(df2,indicator=True,how='outer')
diff1 = diff[diff['_merge'] == 'left_only']

diff1
df1.merge(df2, how='outer', suffixes=['', '_'], indicator=True)
#set(df1.columns).symmetric_difference(df2.columns)

,name,email,prof,_merge
0,ab,abc@g,doc,left_only
1,cd,cde@x,engg,left_only
2,ef,efg@y,dan,both
3,ab,abc@g,pro,right_only
4,cd,cd@x,doc,right_only


In [229]:
head = [['nationality', 'BART', 'USA']]
body = [['placeOfBirth', 'BART', 'NEWYORK'], ['hasFather', 'BART', 'HOMMER']]
head_df = pd.DataFrame(head, columns = ['pred', 'subj', 'obj'])
body_df = pd.DataFrame(body, columns = ['pred', 'subj', 'obj'])
joined_df = pd.merge(head_df, body_df, how='outer')
joined_df

,pred,subj,obj
0,nationality,BART,USA
1,placeOfBirth,BART,NEWYORK
2,hasFather,BART,HOMMER


In [232]:
joined_df = pd.concat([head_df, body_df], join='outer', axis=1)
joined_df

,pred,subj,obj,pred,subj,obj
0,nationality,BART,USA,placeOfBirth,BART,NEWYORK
1,NaN,NaN,NaN,hasFather,BART,HOMMER


In [233]:
pd.__version__

'1.2.3'

In [5]:
d="""full path,name,time
C:\\Users\\User\\Desktop\\Test1\\1.txt,1.txt,10:20
C:\\Users\\User\\Desktop\\Test1\\1.txt,1.txt,10:25
C:\\Users\\User\\Desktop\\Test1\\Test2\\1.txt,1.txt,10:30
C:\\Users\\User\\Desktop\\Test1\\1.txt,1.txt,10:40
C:\\Users\\User\\Desktop\\Test1\\2.txt,2.txt,10:50
C:\\Users\\User\\Desktop\\Test1\\Test2\\1.txt,2.txt,10:60"""
df=g(d)
df

,full path,name,time
0,C:\Users\User\Desktop\Test1\1.txt,1.txt,10:20
1,C:\Users\User\Desktop\Test1\1.txt,1.txt,10:25
2,C:\Users\User\Desktop\Test1\Test2\1.txt,1.txt,10:30
3,C:\Users\User\Desktop\Test1\1.txt,1.txt,10:40
4,C:\Users\User\Desktop\Test1\2.txt,2.txt,10:50
5,C:\Users\User\Desktop\Test1\Test2\1.txt,2.txt,10:60


In [18]:
df.groupby(['full path','name']).shift().fillna(0,limit=1)

,time
0,0
1,10:20
2,NaN
3,10:25
4,NaN
5,NaN


In [48]:
data = pd.DataFrame({"id": [1, 2, 3, 4, 5],
                     "end_time": ["2016-01-13", "2016-01-01", "2016-11-12", "2016-01-17", "2016-03-13"]})
data

,id,end_time
0,1,2016-01-13
1,2,2016-01-01
2,3,2016-11-12
3,4,2016-01-17
4,5,2016-03-13


In [49]:
#data['end'] = pd.to_datetime(data.end_time) 
#data
data["end"] = data["end_time"].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   id        5 non-null      int64         
 1   end_time  5 non-null      object        
 2   end       5 non-null      datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 248.0+ bytes


In [76]:
d="""Evenid,memberid,response
932,48462,0
932,65542,1
947,48462,0
947,56494,0
947,62049,0"""
df=g(d)
df

,Evenid,memberid,response
0,932,48462,0
1,932,65542,1
2,947,48462,0
3,947,56494,0
4,947,62049,0


In [77]:

df['Evenid'] = df.Evenid[~df.Evenid.duplicated()]
df

,Evenid,memberid,response
0,932.0,48462,0
1,NaN,65542,1
2,947.0,48462,0
3,NaN,56494,0
4,NaN,62049,0


In [45]:
df1 = DataFrame({
    'keyword':['abc', 'def', 'ghi']
})

df2 = DataFrame({
    'text to check':['def', 'abcd', 'ghi']
})



In [44]:
df1[df1.keyword.isin(df2['text to check'])]

,keyword
1,def
2,ghi


In [78]:
df = pd.DataFrame({'col1': ['Hello, world. Good day','My name is Bob. Call Me','good evening','yep. stack Overflow.',"Ain't McDonald Yo"]})
df

,col1
0,"Hello, world. Good day"
1,My name is Bob. Call Me
2,good evening
3,yep. stack Overflow.
4,Ain't McDonald Yo


In [91]:
for i in df.col1.str.split(" ").str[1:]:
    [print(s.istitle()) for s in i]

False
True
False
False
False
True
True
True
False
False
True
False
True


In [83]:
'abc'.capitalize()

'Abc'

In [26]:
d="""colA,colB,colC
0.4,2,test1
0.7,3,test2
0.8,4,test3"""
df=g(d)
df

,colA,colB,colC
0,0.4,2,test1
1,0.7,3,test2
2,0.8,4,test3


In [23]:
'colA'+df['colA'].astype(str)+' colB'+df['colB'].astype(str)+' colC'+df['colC'].astype(str)

0    colA0.4 colB2 colCtest1
1    colA0.7 colB3 colCtest2
2    colA0.8 colB4 colCtest3
dtype: object

In [78]:
d="""DATE,EVENT
01/02/2019,x has been attacked
01/02/2019,y has been attacked
01/02/2019,x attacked
03/02/2019,z attacked
03/02/2019,f attacked
04/02/2019,a attacked
05/02/2019,l attacked
05/02/2019,q attacked"""
df=pd.read_csv(StringIO(d))
df
df['DATE']  = pd.to_datetime(df.DATE, dayfirst=True)
#df.groupby([pd.Grouper(key='DATE')]).count().reset_index().rename(columns={'EVENT':'Number of Events'})
#df['DATE']  = pd.to_datetime(df.DATE, dayfirst=True)
#df.resample('D',on='DATE').count().drop('DATE',1).reset_index().rename(columns={'EVENT':'Number of Events'})
df.groupby('DATE').count().reset_index().rename(columns={'EVENT':'Number of Events'})

,DATE,Number of Events
0,2019-02-01,3
1,2019-02-03,2
2,2019-02-04,1
3,2019-02-05,2


In [76]:
df.set_index('DATE').resample('D').count().reset_index().rename(columns={'EVENT':'Number of Events'})

,DATE,Number of Events
0,2019-02-01,3
1,2019-02-02,0
2,2019-02-03,2
3,2019-02-04,1
4,2019-02-05,2


In [116]:
import re
l = ["VISTA AES TIETE E UNT N2 600.",
"VISTA IT AUUNIBANCO PN N1 1.400",
"OPCAO DE VENDA 04/21 COGNP450ON 4,50COGNE 100.000"]



In [118]:
df=DataFrame({
    'col':l
})
df.col.str.extract('(\d*\.*\d*)?$').astype(str).replace('\.','', regex=True)

,0
0,nan
1,1400
2,100000


In [155]:
d="""id,coord_id,val1,val2,record,val3
snp,chr15_1-1000,1.0,0.9,xx12,2
snv,chr15_1-1000,1.0,0.7,yy12,-4
ins,chr15_1-1000,0.01,0.7,jj12,-4
ins,chr15_1-1000,1.0,1.5,zzy1,-5
ins,chr15_1-1000,1.0,1.5,zzy1,-5
del,chr10_2000-4000,0.1,1.2,j112,12
del,chr10_2000-4000,0.4,1.1,jh12,15"""
df=g(d)
df

,id,coord_id,val1,val2,record,val3
0,snp,chr15_1-1000,1.00,0.9,xx12,2
1,snv,chr15_1-1000,1.00,0.7,yy12,-4
2,ins,chr15_1-1000,0.01,0.7,jj12,-4
3,ins,chr15_1-1000,1.00,1.5,zzy1,-5
4,ins,chr15_1-1000,1.00,1.5,zzy1,-5
5,del,chr10_2000-4000,0.10,1.2,j112,12
6,del,chr10_2000-4000,0.40,1.1,jh12,15


In [160]:
df.stack().unstack(1)

,id,coord_id,val1,val2,record,val3
0,snp,chr15_1-1000,1.0,0.9,xx12,2
1,snv,chr15_1-1000,1.0,0.7,yy12,-4
2,ins,chr15_1-1000,0.01,0.7,jj12,-4
3,ins,chr15_1-1000,1.0,1.5,zzy1,-5
4,ins,chr15_1-1000,1.0,1.5,zzy1,-5
5,del,chr10_2000-4000,0.1,1.2,j112,12
6,del,chr10_2000-4000,0.4,1.1,jh12,15


In [162]:
d="""Col1,Col2
A,1
B,1
A,2
A,7
B,6"""
df=pd.read_csv(StringIO(d))
df

,Col1,Col2
0,A,1
1,B,1
2,A,2
3,A,7
4,B,6


In [163]:
#df['num less than 5'] = 
df1 = df[df.groupby('Col1').Col2.apply(lambda x: x<5)].groupby('Col1').agg('max').reset_index()
df2 = df.groupby('Col1', as_index=False).Col2.agg(['count', 'min','max']).reset_index()
df = df1.merge(df2, on='Col1').rename(columns= {
    'Col2':'numbers_under_5',
    'count':'total_numbers',
    'min':'lowest',
    'max':'highest'
})
df

,Col1,numbers_under_5,total_numbers,lowest,highest
0,A,2,3,1,7
1,B,1,2,1,6


In [215]:
from dateutil.parser import parse

df=pd.DataFrame({
    'Date':['20201203', '05/06/20', '2019-09-15 00:00:1568480400']
})
df.Date = df.Date.str.split(' ',1).str[0]
df

,Date
0,20201203
1,05/06/20
2,2019-09-15


In [216]:
df["Date"] = pd.to_datetime(df["Date"]).dt.strftime("%Y-%m-%d")
df

,Date
0,2020-12-03
1,2020-05-06
2,2019-09-15


In [203]:
df['Date'] = df['Date'].apply(parse).dt.strftime("%Y-%m-%d")
df

,Date
0,2020-12-03
1,2020-05-06
2,2019-09-15


In [212]:
for i in range(len(df['Date'])):
    df['Date'][i] = parse(df['Date'][i])
df['Date'] = pd.to_datetime(df['Date']).dt.strftime("%Y-%m-%d")
df

,Date
0,2020-12-03
1,2020-05-06
2,2019-09-15


In [244]:
d="""date
2018-01-01
2018-01-01
2018-01-01
2021-01-08
2021-01-08"""
df=pd.read_csv(StringIO(d))
df


,date
0,2018-01-01
1,2018-01-01
2,2018-01-01
3,2021-01-08
4,2021-01-08


In [262]:
x in pd.to_datetime(df.date).dt.date.values

True

In [254]:
df.loc[pd.to_datetime(df.date) == pd.Timestamp('2018.1.1')]
df.loc[pd.to_datetime(df.date) == '2018.1.1']
df.loc[pd.to_datetime(df.date).dt.date == datetime.date(2018, 1, 1)]
df[pd.to_datetime(df.date) == pd.Timestamp('2018.1.1')]
df[pd.to_datetime(df.date) == '2018.1.1']
df.loc[pd.to_datetime(df.date).dt.date == datetime.date(2018, 1, 1)]

,date
0,2018-01-01
1,2018-01-01
2,2018-01-01


In [224]:
x = datetime.date(2018, 1, 1)
print(type(x))
x in pd.to_datetime(df.date).dt.date

<class 'datetime.date'>


False

In [230]:
x in pd.to_datetime(df.date)
pd.Timestamp('2018.1.1') == pd.to_datetime(df.date)

0     True
1     True
2     True
3     True
4     True
5    False
6    False
Name: date, dtype: bool

In [238]:
(pd.to_datetime(df.date) == pd.Timestamp("2018.1.1")).any()
(pd.to_datetime(df.date).dt.date == pd.Timestamp("2018.1.1")).any()


True

In [240]:
x = datetime.date(2018, 1, 1)
(pd.to_datetime(df.date) == x).any()
#(pd.to_datetime(df.date).dt.date == x).any()

False

In [293]:
db_columns_list = ['col1', 'col2', 'col3']
df=DataFrame({
    'col1':[1,2,3],
    'col3':[3,4,5]
})
#df[df.columns & db_columns_list]
df[set(df.columns).intersection(set(db_columns_list))]
df.index=[0,1,np.nan]
df
df[set(df.columns).intersection(set(db_columns_list))]
df[df.columns.intersection(db_columns_list)]

,col1,col3
0.0,1,3
1.0,2,4
NaN,3,5


In [294]:
import io
d="""ItemID, NameID, Users, Days, Pricing, Expiration, Status
370915293,aaaaqqq.abc,0,0,$12,05/10/2021 11:44,Ran
371192969,aaacns.abc,7,0,$12,05/08/2021 09:34,Ran
370905229,aaamix.abc,0,0,$12,05/07/2021 10:32,Ran
371459366,aaapdf.abc,28,0,$12,05/11/2021 12:55,Ran"""
df=pd.read_csv(StringIO(d), sep=',|;')
df

<ipython-input-294-81fd6826a096>:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df=pd.read_csv(StringIO(d), sep=',|;')


,ItemID,NameID,Users,Days,Pricing,Expiration,Status
0,370915293,aaaaqqq.abc,0,0,$12,05/10/2021 11:44,Ran
1,371192969,aaacns.abc,7,0,$12,05/08/2021 09:34,Ran
2,370905229,aaamix.abc,0,0,$12,05/07/2021 10:32,Ran
3,371459366,aaapdf.abc,28,0,$12,05/11/2021 12:55,Ran


In [279]:
d="""pred,subj,obj
nationality,BART,USA"""
head_df = g(d)
head_df

,pred,subj,obj
0,nationality,BART,USA


In [282]:
d="""pred_x,subj_x,obj_x,pred_y,subj_y,obj_y
nationality,BART,USA,placeOfBirth,BART,NEWYORK
nationality,BART,USA,hasFather,BART,HOMMER"""
body_df = g(d)
body_df

,pred_x,subj_x,obj_x,pred_y,subj_y,obj_y
0,nationality,BART,USA,placeOfBirth,BART,NEWYORK
1,nationality,BART,USA,hasFather,BART,HOMMER


In [363]:
d="""Date,Train,Station,Time,ID
02.10.2020,Flixtrain,London,10:40,1
02.10.2020,Flixtrain,Berlin,20:30,1
02.10.2020,Flixtrain,Hamburg,23:45,1
02.10.2020,VSOE,Amesterdam,21:30,2
02.10.2020,VSOE,Cologne,00:50,2
02.10.2020,VSOE,Berlin,04:30,2
02.10.2020,ICE-220,Warschau,12:35,3
02.10.2020,ICE-220,Breslau,17:40,3
02.10.2020,ICE-220,Prag,23:13,3
02.10.2020,ICE-342,Wien,00:35,4
02.10.2020,ICE-342,Salzburg,07:42,4
02.10.2020,ICE-342,Munich,13:13,4"""
df=g(d)
df

,Date,Train,Station,Time,ID
0,02.10.2020,Flixtrain,London,10:40,1
1,02.10.2020,Flixtrain,Berlin,20:30,1
2,02.10.2020,Flixtrain,Hamburg,23:45,1
3,02.10.2020,VSOE,Amesterdam,21:30,2
4,02.10.2020,VSOE,Cologne,00:50,2
5,02.10.2020,VSOE,Berlin,04:30,2
6,02.10.2020,ICE-220,Warschau,12:35,3
7,02.10.2020,ICE-220,Breslau,17:40,3
8,02.10.2020,ICE-220,Prag,23:13,3
9,02.10.2020,ICE-342,Wien,00:35,4


In [364]:
df['t'] = pd.to_datetime(df.Time, format='%H:%M')
df.Date = pd.to_datetime(df.Date, dayfirst=True)

cond = df.groupby(['Train','ID']
                 ).t.transform('first') - df.groupby(
    ['Train','ID']).t.transform(
          lambda x: x.values) > pd.Timedelta('0 days')

df.Date = df.Date.mask(cond,df.Date + pd.Timedelta(days=1))
df = df.drop('t',1)
df


,Date,Train,Station,Time,ID
0,2020-10-02,Flixtrain,London,10:40,1
1,2020-10-02,Flixtrain,Berlin,20:30,1
2,2020-10-02,Flixtrain,Hamburg,23:45,1
3,2020-10-02,VSOE,Amesterdam,21:30,2
4,2020-10-03,VSOE,Cologne,00:50,2
5,2020-10-03,VSOE,Berlin,04:30,2
6,2020-10-02,ICE-220,Warschau,12:35,3
7,2020-10-02,ICE-220,Breslau,17:40,3
8,2020-10-02,ICE-220,Prag,23:13,3
9,2020-10-02,ICE-342,Wien,00:35,4


In [369]:
data_df = pd.DataFrame({
    "Reference": ("A", "A", "A", "B", "C", "C", "D", "E"),
    "Value1": ("U", "U", "U--","V", "W", "W--", "X", "Y"),
    "Value2": ("u", "u--", "u","v", "w", "w", "x", "y")
    }, index=[1, 2, 3, 4, 5, 6, 7, 8])
truth_df = pd.DataFrame({
    "Reference": ("A", "B", "C", "D", "E"),
    "Value1": ("U", "V", "W", "X", "Y"),
    "Value2": ("u", "v", "w", "x", "y")
    }, index=[1, 4, 5, 7, 8])
data_df

,Reference,Value1,Value2
1,A,U,u
2,A,U,u--
3,A,U--,u
4,B,V,v
5,C,W,w
6,C,W--,w
7,D,X,x
8,E,Y,y


In [370]:
truth_df

,Reference,Value1,Value2
1,A,U,u
4,B,V,v
5,C,W,w
7,D,X,x
8,E,Y,y


In [373]:
data_df.filter(like='Value').isin(truth_df.filter(like='Value'))

,Value1,Value2
1,True,True
2,False,False
3,False,False
4,True,True
5,True,True
6,False,False
7,True,True
8,True,True


In [368]:
df_out = data_df.merge(truth_df, on=['Reference', 'Value1','Value2'], how='left', indicator=True).rename(columns={"_merge":"Issues"})
#df_out["Issues"] = np.where(df_out["Issues"] == "left_only", "Flag", "")
df_out

,Reference,Value1,Value2,Issues
0,A,U,u,both
1,A,U,u--,left_only
2,A,U--,u,left_only
3,B,V,v,both
4,C,W,w,both
5,C,W--,w,left_only
6,D,X,x,both
7,E,Y,y,both


In [376]:
d="""buying date,selling date
2000-01-04,2000-04-15"""
df=g(d)
df['buying date']=pd.to_datetime(df['buying date'])
df['selling date']=pd.to_datetime(df['selling date'])
df

,buying date,selling date
0,2000-01-04,2000-04-15


In [ ]:
df_new=DataFrame(
    'Date':pd.to_datetime(start=df['buying date'], 
                          end=df['selling date'])
)

In [6]:
rdf = pd.DataFrame({'A':[10,20]},index=pd.date_range(start='2020-05-04 08:00:00', freq='1h', periods=2))
df2 = pd.DataFrame({'A':[30,40]},index=pd.date_range(start='2020-05-04 10:00:00', freq='1h', periods=2))
rdf

,A
2020-05-04 08:00:00,10
2020-05-04 09:00:00,20


In [7]:
df2

,A
2020-05-04 10:00:00,30
2020-05-04 11:00:00,40


In [8]:
rdf= pd.concat([rdf,df2])
rdf

,A
2020-05-04 08:00:00,10
2020-05-04 09:00:00,20
2020-05-04 10:00:00,30
2020-05-04 11:00:00,40


In [9]:
rdf= pd.concat([rdf,df2])
rdf

,A
2020-05-04 08:00:00,10
2020-05-04 09:00:00,20
2020-05-04 10:00:00,30
2020-05-04 11:00:00,40
2020-05-04 10:00:00,30
2020-05-04 11:00:00,40


In [10]:
rdf.drop_duplicates(keep='first',inplace=True)

In [11]:
rdf

,A
2020-05-04 08:00:00,10
2020-05-04 09:00:00,20
2020-05-04 10:00:00,30
2020-05-04 11:00:00,40
